In [27]:
from ipywidgets import interact
import numpy as np
from bokeh.layouts import row, column
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Range1d
import time

from bokeh.palettes import Dark2_5 as palette
# itertools handles the cycling
import itertools  

colors = itertools.cycle(palette)


output_notebook()

Loading BokehJS ...

In [28]:
t0 = time.time()

import zmq
from silayer.raw2hdf import DataPacket

def loop(data_socket):
    while True:
        data = data_socket.recv() ## This should block until data shows up
        dp = DataPacket(data)
        ## Then do whatever...
        
def main(host="si-layer.local", data_port=9998):
    ctx = zmq.Context()
    data_addr = f"tcp://{host}:{data_port}"
    data_socket = ctx.socket(zmq.SUB)
    data_socket.setsockopt_string(zmq.SUBSCRIBE, "")
    data_socket.connect(data_addr)
    
    return data_socket

socket = main("10.10.0.11")

In [29]:
p = figure(title='ASIC_<x>', tools='', 
       plot_height=350, plot_width=800,
       background_fill_color="#fafafa", )
p.xaxis.axis_label = "channel"
p.yaxis.axis_label = "adc counts (LSB)"
#p.y_range = Range1d(0, 400)

n_ch = 32
channels = np.array(range(n_ch)) + .5

data_dict = {'pedestals': np.zeros(n_ch) -100,
             'channels': channels
            }

data = ColumnDataSource(data_dict)
steps = p.step('channels', 'pedestals', source=data)

channel_stream = figure(title='ASIC_<x>', tools='', 
       plot_height=350, plot_width=800,
       background_fill_color="#fafafa", )

channel_stream.xaxis.axis_label = "event number"
channel_stream.yaxis.axis_label = "adc counts (LSB)"


data = {'time': np.zeros(0)}
for ch in range(3):
    data[f'ch{ch:02d}'] = []
    
datastream = ColumnDataSource(data=data)

for ch in range(3):
    color=next(colors)
    channel_stream.line('time', f'ch{ch:02d}', source=datastream, color=color)
    channel_stream.circle('time', f'ch{ch:02d}', source=datastream, legend_label=f'ch{ch:02d}', color=color)


In [30]:
t = show(column(p, channel_stream), notebook_handle=True)
#ts_handle = show(channel_stream, notebook_handle=True)

In [32]:
event_counter = 5000
for i in range(200):
    event_counter += 1
    
    data = 250 + np.random.normal(scale=5, size=n_ch)
    steps.data_source.data['pedestals'] = data
    updated_data = {'time': [event_counter]}
    for ch in range(3):
        updated_data[f'ch{ch:02d}'] = [np.random.normal(scale=5) + 10*ch]
        
    datastream.stream(updated_data, rollover=100)    
    

    push_notebook(t)
    
    time.sleep(np.abs(np.random.normal(0.05, scale=0.01)))

In [ ]:
#for i in range(100):
i = 0
n_received = 0
while True:
    data = socket.recv()
    n_received += 1
    i += 1
    if i % 200 == 0 and i != 0:
        i = 0
        dp = DataPacket(data)
        ap = dp.asic_packets[0]

        steps.data_source.data['pedestals']  = ap.data

        
        datastream.stream({'time': [dp.event_counter], 
                           'adc0':[ap.data[0]], 
                           'adc1':[ap.data[1]]}, 
                           rollover=1000)    
        #datastream.stream({'time': np.zeros(32) + dp.event_counter, 
        #                   'adc': ap.data}, rollover=100)    
    
    push_notebook(t)